In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
vid = pd.read_csv('../input/NFL-Punt-Analytics-Competition/video_review.csv')
vid.head(10)

In [ ]:
game = pd.read_csv('../input/NFL-Punt-Analytics-Competition/game_data.csv')
game.head(10)

In [ ]:
game['concussion'] = np.isin(game['GameKey'], vid['GameKey'])
game.head(10)

In [ ]:
prole = pd.read_csv('../input/NFL-Punt-Analytics-Competition/play_player_role_data.csv')
prole.head(10)

In [ ]:
prole['concussion'] = prole['concussion'] = np.isin(prole['GameKey'], vid['GameKey'])
prole.head(10)

In [ ]:
pinfo = pd.read_csv('../input/NFL-Punt-Analytics-Competition/play_information.csv')
pinfo.head(10)

In [ ]:
pinfo['concussion'] = np.isin(pinfo['GameKey'], vid['GameKey'])
pinfo.head(10)

In [ ]:
dist = pinfo['PlayDescription'].str.split(pat='punts', n=1, expand=True)
pinfo['punt_dist'] = dist[1].str.extract('(\d+)')
pinfo['punt_dist'] = pinfo['punt_dist'].fillna(0)
pinfo['punt_dist'] = pd.to_numeric(pinfo['punt_dist'])
pinfo.head(10)

In [ ]:
ngs_reg_2017_1_6 = pd.read_csv('../input/NFL-Punt-Analytics-Competition/NGS-2016-reg-wk1-6.csv')
ngs_reg_2017_7_12 = pd.read_csv('../input/NFL-Punt-Analytics-Competition/NGS-2016-reg-wk7-12.csv')
ngs_reg_2017_13_17 = pd.read_csv('../input/NFL-Punt-Analytics-Competition/NGS-2016-reg-wk13-17.csv')

ngs_reg_2017 = pd.concat([ngs_reg_2017_1_6,ngs_reg_2017_7_12,ngs_reg_2017_13_17])
ngs_reg_2017.head()


In [ ]:
ngs_reg_2017['concussion'] = np.isin(ngs_reg_2017['GameKey'], vid['GameKey'])
ngs_reg_2017.head()

**Univariant Analysis**

In [ ]:
from matplotlib import figure
fig = figure.Figure(figsize=(10, 7.5))
sns.countplot(x='Quarter', data=pinfo[pinfo['concussion'] == True], color='blue')

In [ ]:
game['Game_Start_Hour'] = game['Start_Time'].str.split(':').map(lambda x: x[0])
game.head()

In [ ]:
fig = figure.Figure(figsize=(10, 7.5))
sns.countplot(x='Game_Start_Hour', data=game[game['concussion'] == True], color='blue')

In [ ]:
f, x = plt.subplots(1,2, figsize=(16, 7.6))
sns.countplot(x='Primary_Impact_Type', data=vid, color='orange', ax=x[0])
sns.countplot(x='Player_Activity_Derived', data=vid, color='blue', ax=x[1])

In [ ]:
f = figure.Figure(figsize=(16, 7.5))
sns.boxplot(x=ngs_reg_2017[ngs_reg_2017['concussion'] == True]['x'])

In [ ]:
sns.violinplot(x=ngs_reg_2017[ngs_reg_2017['concussion'] == True]['x'])

In [ ]:
set(game['Turf'].values)

In [ ]:
rep = {
    'Nat': 'Grass',
    'Grass': 'Grass',
    'gras': 'Grass',
    'Arti': 'Artificial',
    'Turf': 'Turf',
    'turf': 'Turf',
    'UBU': 'Ubu Speed',
    'Synthetic': 'Synthetic'
}

clean_turf = np.array([], str)

for value in game['Turf'].values:
    for k in list(rep.keys()):
        if isinstance(value, str) and k in value:
            clean_turf = np.append(clean_turf, rep[k])
            break
        elif value is np.nan:
            clean_turf = np.append(clean_turf, np.nan)
            break
            
len(clean_turf)

In [ ]:
game['clean_turf'] = clean_turf

In [ ]:
sns.countplot(x='clean_turf', data=game[game['concussion'] == True], color='blue')

In [ ]:
prole['concussion'] = prole['concussion'].replace([True, False], [1, 0])
pr = prole.groupby(['Role'], as_index=False).agg({'concussion': 'sum'})
pr.sort_values(by=['concussion'], inplace=True, ascending=False)
pr

In [ ]:
import squarify
cmap = matplotlib.cm.Blues
my_values = pr[pr['concussion'] != 0]['concussion']
mini=min(my_values)
maxi=max(my_values)
norm = matplotlib.colors.Normalize(vmin=mini, vmax=maxi)
colors = [cmap(norm(value)) for value in my_values]

plt.figure(figsize=(10, 10))
squarify.plot(sizes=pr[pr['concussion'] != 0]['concussion'], label=pr['Role'], alpha=0.5, color=colors)
plt.axis('off')
plt.show()

In [ ]:
sns.countplot(x='Quarter', hue='concussion', data=pinfo)

In [ ]:
ax = sns.countplot(x='Player_Activity_Derived', hue='Friendly_Fire', data=vid)

In [ ]:
sns.countplot(x='Game_Start_Hour', data=game, hue='concussion')

In [ ]:
sns.countplot(x='clean_turf', data=game, hue='concussion')

In [ ]:
surface_proportion = game.groupby(['clean_turf', 'concussion']).agg({'GameKey': 'count'})
surface_proportion

In [ ]:
surface_proportion.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

In [ ]:
set(ngs_reg_2017[ngs_reg_2017['concussion'] == True]['GameKey'])

In [ ]:
hm = ngs_reg_2017[ngs_reg_2017['GameKey'] == 473]
hm = hm[['x', 'y', 'concussion']]
hm['x'] = hm['x'].astype(int).astype('category')
hm['y'] = hm['y'].astype(int).astype('category')
hm.head()

In [ ]:
hm['concussion'] = hm['concussion'].replace([True, False], [1, 0])
hm.head()